# 第3章: 統計学基礎

この章では、simple-statisticsを使って記述統計の基本を深く学びます。

## 学習目標
- 代表値（平均、中央値、最頻値）の理解
- 散布度（分散、標準偏差、四分位範囲）の理解
- 歪度と尖度
- 相関係数
- データの標準化

In [1]:
// ライブラリの読み込み
const math = await import('https://esm.sh/mathjs@13.0.0');
const ss = await import('https://esm.sh/simple-statistics@7.8.8');
console.log('Libraries loaded!');

Libraries loaded!


## 3.1 データの種類

統計学では、データを以下のように分類します：

- **質的データ（カテゴリカルデータ）**
  - 名義尺度: 性別、血液型など
  - 順序尺度: 満足度（高・中・低）、学年など

- **量的データ（数値データ）**
  - 間隔尺度: 温度、年代など（0に意味がない）
  - 比例尺度: 身長、体重、金額など（0が絶対的な意味を持つ）

## 3.2 代表値（中心傾向の指標）

代表値は、データ全体を1つの値で表現するものです。

### 3.2.1 算術平均（Arithmetic Mean）

In [2]:
const salaries = [300, 320, 350, 380, 400, 450, 500, 550, 600, 2000];

console.log('給与データ（万円）:', salaries);

// 算術平均
const arithmeticMean = ss.mean(salaries);
console.log('\n算術平均:', arithmeticMean, '万円');

// 手計算で確認
const sum = salaries.reduce((a, b) => a + b, 0);
console.log('検算: 合計', sum, '÷', salaries.length, '=', sum / salaries.length);

console.log('\n注意: 極端な値（2000万円）があると平均が大きく影響を受けます');

給与データ（万円）: [
   300, 320, 350,
   380, 400, 450,
   500, 550, 600,
  2000
]



算術平均: 585 万円


検算: 合計 5850 ÷ 10 = 585



注意: 極端な値（2000万円）があると平均が大きく影響を受けます


### 3.2.2 中央値（Median）

In [3]:
const salaries2 = [300, 320, 350, 380, 400, 450, 500, 550, 600, 2000];

// 中央値
const medianValue = ss.median(salaries2);
console.log('中央値:', medianValue, '万円');

// ソートして確認
const sorted = [...salaries2].sort((a, b) => a - b);
console.log('ソート済み:', sorted);
console.log('中央の2つ:', sorted[4], 'と', sorted[5]);
console.log('その平均:', (sorted[4] + sorted[5]) / 2);

console.log('\n中央値は外れ値の影響を受けにくい（ロバスト）');

中央値: 425 万円


ソート済み: [
   300, 320, 350,
   380, 400, 450,
   500, 550, 600,
  2000
]


中央の2つ: 400 と 450


その平均: 425



中央値は外れ値の影響を受けにくい（ロバスト）


### 3.2.3 最頻値（Mode）

In [4]:
const scores = [3, 4, 4, 5, 5, 5, 5, 6, 6, 7];

console.log('テストの点数:', scores);

// 最頻値
const modeValue = ss.mode(scores);
console.log('最頻値:', modeValue);

// 度数を確認
const frequency = {};
scores.forEach(s => { frequency[s] = (frequency[s] || 0) + 1; });
console.log('度数:', frequency);

テストの点数: [
  3, 4, 4, 5, 5,
  5, 5, 6, 6, 7
]


最頻値: 5


度数: { "3": 1, "4": 2, "5": 4, "6": 2, "7": 1 }


### 3.2.4 その他の平均

In [5]:
const growthRates = [1.05, 1.10, 1.08, 1.12, 1.06];

console.log('成長率データ:', growthRates);

// 算術平均
console.log('\n算術平均:', ss.mean(growthRates));

// 幾何平均（成長率の平均に適切）
const geometricMean = ss.geometricMean(growthRates);
console.log('幾何平均:', geometricMean);

// 調和平均（速度の平均に適切）
const harmonicMean = ss.harmonicMean(growthRates);
console.log('調和平均:', harmonicMean);

// 二乗平均（RMS）
const rms = ss.rootMeanSquare(growthRates);
console.log('二乗平均平方根:', rms);

console.log('\n関係: 調和平均 ≤ 幾何平均 ≤ 算術平均 ≤ 二乗平均');

成長率データ: [ 1.05, 1.1, 1.08, 1.12, 1.06 ]



算術平均: 1.082


幾何平均: 1.0816977479614276


調和平均: 1.0813964614927878


二乗平均平方根: 1.0823030998754464



関係: 調和平均 ≤ 幾何平均 ≤ 算術平均 ≤ 二乗平均


### 3.2.5 トリム平均

In [6]:
const dataWithOutliers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 100];

console.log('外れ値を含むデータ:', dataWithOutliers);
console.log('\n通常の平均:', ss.mean(dataWithOutliers));
console.log('中央値:', ss.median(dataWithOutliers));

// トリム平均（上下10%を除外）
// simple-statisticsにはないので手動で実装
function trimmedMean(arr, trimPercent) {
    const sorted = [...arr].sort((a, b) => a - b);
    const trimCount = Math.floor(sorted.length * trimPercent);
    const trimmed = sorted.slice(trimCount, sorted.length - trimCount);
    return ss.mean(trimmed);
}

console.log('10%トリム平均:', trimmedMean(dataWithOutliers, 0.1));
console.log('20%トリム平均:', trimmedMean(dataWithOutliers, 0.2));

外れ値を含むデータ: [
  1, 2, 3, 4,   5,
  6, 7, 8, 9, 100
]



通常の平均: 14.5


中央値: 5.5


10%トリム平均: 5.5


20%トリム平均: 5.5


## 3.3 散布度（ばらつきの指標）

### 3.3.1 範囲（Range）

In [7]:
const temps = [15, 18, 22, 25, 20, 17, 23, 28, 19, 21];

console.log('気温データ:', temps);

const minVal = ss.min(temps);
const maxVal = ss.max(temps);
const range = maxVal - minVal;

console.log('最小値:', minVal);
console.log('最大値:', maxVal);
console.log('範囲:', range);

気温データ: [
  15, 18, 22, 25, 20,
  17, 23, 28, 19, 21
]


最小値: 15


最大値: 28


範囲: 13


### 3.3.2 分散と標準偏差

In [8]:
const heights = [165, 170, 168, 172, 175, 169, 171, 167, 173, 170];

console.log('身長データ (cm):', heights);
console.log('平均:', ss.mean(heights));

// 母分散（N で割る）
console.log('\n母分散:', ss.variance(heights));

// 標本分散（N-1 で割る、不偏推定量）
console.log('標本分散:', ss.sampleVariance(heights));

// 標準偏差
console.log('\n母標準偏差:', ss.standardDeviation(heights));
console.log('標本標準偏差:', ss.sampleStandardDeviation(heights));

console.log('\n注: サンプルデータから母集団を推定する場合は標本分散/標本標準偏差を使用');

身長データ (cm): [
  165, 170, 168, 172,
  175, 169, 171, 167,
  173, 170
]


平均: 170



母分散: 7.8


標本分散: 8.666666666666666



母標準偏差: 2.792848008753788


標本標準偏差: 2.943920288775949



注: サンプルデータから母集団を推定する場合は標本分散/標本標準偏差を使用


### 分散の計算過程を理解する

In [9]:
const data3 = [2, 4, 6, 8, 10];
const mean3 = ss.mean(data3);

console.log('データ:', data3);
console.log('平均:', mean3);
console.log('');

// 各データと平均との差（偏差）
console.log('偏差の計算:');
const deviations = data3.map(x => {
    const dev = x - mean3;
    console.log(`  ${x} - ${mean3} = ${dev}`);
    return dev;
});

// 偏差の2乗
console.log('\n偏差の2乗:');
const squaredDeviations = deviations.map(d => {
    const sq = d * d;
    console.log(`  ${d}² = ${sq}`);
    return sq;
});

// 分散（母分散）
const sumSquared = squaredDeviations.reduce((a, b) => a + b, 0);
const n3 = data3.length;
console.log('\n偏差2乗の合計:', sumSquared);
console.log('母分散 (÷n):', sumSquared / n3);
console.log('標本分散 (÷(n-1)):', sumSquared / (n3 - 1));

データ: [ 2, 4, 6, 8, 10 ]


平均: 6


偏差の計算:


  2 - 6 = -4


  4 - 6 = -2


...

  2² = 4


  4² = 16



偏差2乗の合計: 40


母分散 (÷n): 8


標本分散 (÷(n-1)): 10


### 3.3.3 四分位範囲（IQR）

In [10]:
const testData = [23, 25, 28, 30, 32, 35, 38, 40, 42, 45, 48, 50, 55, 60, 65, 70, 75, 80, 85, 95];

console.log('データ:', testData);
console.log('データ数:', testData.length);

// 四分位数
const q1 = ss.quantile(testData, 0.25);
const q2 = ss.quantile(testData, 0.50);  // = 中央値
const q3 = ss.quantile(testData, 0.75);

console.log('\n第1四分位数 (Q1, 25%):', q1);
console.log('第2四分位数 (Q2, 50%, 中央値):', q2);
console.log('第3四分位数 (Q3, 75%):', q3);

// 四分位範囲
const iqr = ss.interquartileRange(testData);
console.log('\n四分位範囲 (IQR = Q3 - Q1):', iqr);

// 外れ値の検出（IQRの1.5倍ルール）
const lowerBound = q1 - 1.5 * iqr;
const upperBound = q3 + 1.5 * iqr;
console.log('\n外れ値の境界:');
console.log('  下限:', lowerBound);
console.log('  上限:', upperBound);

const outliers = testData.filter(x => x < lowerBound || x > upperBound);
console.log('外れ値:', outliers.length > 0 ? outliers : 'なし');

データ: [
  23, 25, 28, 30, 32, 35, 38,
  40, 42, 45, 48, 50, 55, 60,
  65, 70, 75, 80, 85, 95
]


データ数: 20



第1四分位数 (Q1, 25%): 33.5


第2四分位数 (Q2, 50%, 中央値): 46.5


第3四分位数 (Q3, 75%): 67.5



四分位範囲 (IQR = Q3 - Q1): 34



外れ値の境界:


  下限: -17.5


  上限: 118.5


外れ値: なし


### 3.3.4 変動係数（CV）

In [11]:
// 異なる単位のデータを比較する
const heights2 = [165, 170, 168, 172, 175];  // cm
const weights = [55, 70, 62, 78, 85];        // kg

console.log('身長 (cm):', heights2);
console.log('体重 (kg):', weights);

// 変動係数 = 標準偏差 / 平均 × 100 (%)
const cvHeight = (ss.standardDeviation(heights2) / ss.mean(heights2)) * 100;
const cvWeight = (ss.standardDeviation(weights) / ss.mean(weights)) * 100;

console.log('\n身長の変動係数:', cvHeight.toFixed(2), '%');
console.log('体重の変動係数:', cvWeight.toFixed(2), '%');
console.log('\n体重の方がばらつきが大きい');

身長 (cm): [ 165, 170, 168, 172, 175 ]


体重 (kg): [ 55, 70, 62, 78, 85 ]



身長の変動係数: 2.00 %


体重の変動係数: 15.36 %



体重の方がばらつきが大きい


## 3.4 分布の形状

### 3.4.1 歪度（Skewness）

In [12]:
// 対称な分布
const symmetric = [1, 2, 3, 4, 5, 6, 7, 8, 9];

// 右に歪んだ分布（正の歪度）
const rightSkewed = [1, 1, 2, 2, 2, 3, 3, 5, 10];

// 左に歪んだ分布（負の歪度）
const leftSkewed = [1, 6, 8, 8, 8, 9, 9, 10, 10];

console.log('歪度の計算:');
console.log('\n対称なデータ:', symmetric);
console.log('  歪度:', ss.sampleSkewness(symmetric).toFixed(4));

console.log('\n右に歪んだデータ:', rightSkewed);
console.log('  歪度:', ss.sampleSkewness(rightSkewed).toFixed(4));
console.log('  平均:', ss.mean(rightSkewed).toFixed(2), '> 中央値:', ss.median(rightSkewed));

console.log('\n左に歪んだデータ:', leftSkewed);
console.log('  歪度:', ss.sampleSkewness(leftSkewed).toFixed(4));
console.log('  平均:', ss.mean(leftSkewed).toFixed(2), '< 中央値:', ss.median(leftSkewed));

歪度の計算:



対称なデータ: [
  1, 2, 3, 4, 5,
  6, 7, 8, 9
]


  歪度: 0.0000



右に歪んだデータ: [
  1, 1, 2,  2, 2,
  3, 3, 5, 10
]


  歪度: 2.0780


  平均: 3.22 > 中央値: 2



左に歪んだデータ: [
  1, 6,  8,  8, 8,
  9, 9, 10, 10
]


  歪度: -2.0162


  平均: 7.67 < 中央値: 8


### 3.4.2 尖度（Kurtosis）

In [13]:
// 正規分布に近いデータ
const normal = [2, 3, 4, 4, 5, 5, 5, 6, 6, 7, 8];

// 尖った分布
const leptokurtic = [4, 4.5, 4.8, 5, 5, 5, 5, 5.2, 5.5, 6];

// 平坦な分布
const platykurtic = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];

console.log('尖度の計算（超過尖度）:');
console.log('\n正規分布に近いデータ:', normal);
console.log('  尖度:', ss.sampleKurtosis(normal).toFixed(4));

console.log('\n尖った分布:', leptokurtic);
console.log('  尖度:', ss.sampleKurtosis(leptokurtic).toFixed(4));

console.log('\n平坦な分布:', platykurtic);
console.log('  尖度:', ss.sampleKurtosis(platykurtic).toFixed(4));

console.log('\n注: 正規分布の超過尖度 = 0');
console.log('    尖度 > 0: 尖った分布');
console.log('    尖度 < 0: 平坦な分布');

尖度の計算（超過尖度）:



正規分布に近いデータ: [
  2, 3, 4, 4, 5,
  5, 5, 6, 6, 7,
  8
]


  尖度: -0.1333



尖った分布: [
  4, 4.5, 4.8,   5, 5,
  5,   5, 5.2, 5.5, 6
]


  尖度: 1.3129



平坦な分布: [
  1, 2, 3, 4,  5,
  6, 7, 8, 9, 10
]


  尖度: -1.2000



注: 正規分布の超過尖度 = 0


    尖度 > 0: 尖った分布


    尖度 < 0: 平坦な分布


## 3.5 相関分析

### 3.5.1 共分散

In [14]:
// 勉強時間と点数
const studyHours = [2, 3, 4, 5, 6, 7, 8];
const examScores = [55, 60, 68, 75, 82, 88, 95];

console.log('勉強時間:', studyHours);
console.log('試験の点数:', examScores);

// 共分散
const covariance = ss.sampleCovariance(studyHours, examScores);
console.log('\n共分散:', covariance.toFixed(4));

// 共分散の計算過程
const meanX = ss.mean(studyHours);
const meanY = ss.mean(examScores);
console.log('\n計算過程:');
console.log('勉強時間の平均:', meanX);
console.log('点数の平均:', meanY.toFixed(2));

let sumProduct = 0;
studyHours.forEach((x, i) => {
    const y = examScores[i];
    const prod = (x - meanX) * (y - meanY);
    sumProduct += prod;
});
console.log('偏差積の合計:', sumProduct.toFixed(2));
console.log('共分散 (÷(n-1)):', (sumProduct / (studyHours.length - 1)).toFixed(4));

勉強時間: [
  2, 3, 4, 5,
  6, 7, 8
]


試験の点数: [
  55, 60, 68, 75,
  82, 88, 95
]



共分散: 31.6667



計算過程:


勉強時間の平均: 5


点数の平均: 74.71


偏差積の合計: 190.00


共分散 (÷(n-1)): 31.6667


### 3.5.2 ピアソンの相関係数

In [15]:
// 正の相関
const x1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
const y1 = [2, 4, 5, 4, 5, 7, 8, 9, 10, 11];

// 負の相関
const x2 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
const y2 = [10, 9, 8, 8, 6, 5, 4, 3, 2, 1];

// 無相関
const x3 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10];
const y3 = [5, 3, 7, 4, 6, 5, 8, 2, 6, 4];

console.log('ピアソンの相関係数:');
console.log('\n正の相関: r =', ss.sampleCorrelation(x1, y1).toFixed(4));
console.log('負の相関: r =', ss.sampleCorrelation(x2, y2).toFixed(4));
console.log('無相関: r =', ss.sampleCorrelation(x3, y3).toFixed(4));

console.log('\n相関係数の解釈:');
console.log('  |r| ≥ 0.7: 強い相関');
console.log('  0.4 ≤ |r| < 0.7: 中程度の相関');
console.log('  0.2 ≤ |r| < 0.4: 弱い相関');
console.log('  |r| < 0.2: ほとんど相関なし');

ピアソンの相関係数:



正の相関: r = 0.9755


負の相関: r = -0.9949


無相関: r = -0.0201



相関係数の解釈:


  |r| ≥ 0.7: 強い相関


  0.4 ≤ |r| < 0.7: 中程度の相関


  0.2 ≤ |r| < 0.4: 弱い相関


  |r| < 0.2: ほとんど相関なし


### 3.5.3 決定係数（R²）

In [16]:
const studyX = [2, 3, 4, 5, 6, 7, 8];
const scoreY = [55, 60, 68, 75, 82, 88, 95];

const r = ss.sampleCorrelation(studyX, scoreY);
const rSquared = r * r;

console.log('相関係数 r:', r.toFixed(4));
console.log('決定係数 R²:', rSquared.toFixed(4));
console.log('');
console.log('解釈: 勉強時間で点数の変動の', (rSquared * 100).toFixed(1), '%を説明できる');

相関係数 r: 0.9992


決定係数 R²: 0.9983


解釈: 勉強時間で点数の変動の 99.8 %を説明できる


## 3.6 データの標準化

### 3.6.1 Zスコア（標準得点）

In [17]:
const scores2 = [65, 70, 75, 80, 85, 90, 95];

console.log('元のスコア:', scores2);

const scoreMean = ss.mean(scores2);
const scoreStd = ss.standardDeviation(scores2);

console.log('平均:', scoreMean);
console.log('標準偏差:', scoreStd.toFixed(4));

// Zスコアへ変換: z = (x - μ) / σ
const zScores = scores2.map(x => ss.zScore(x, scoreMean, scoreStd));

console.log('\nZスコア:');
scores2.forEach((score, i) => {
    console.log(`  ${score} → z = ${zScores[i].toFixed(4)}`);
});

// 標準化後のデータの性質
console.log('\n標準化後のデータの性質:');
console.log('  平均:', ss.mean(zScores).toFixed(10), '(≈ 0)');
console.log('  標準偏差:', ss.standardDeviation(zScores).toFixed(10), '(≈ 1)');

元のスコア: [
  65, 70, 75, 80,
  85, 90, 95
]


平均: 80


標準偏差: 10.0000



Zスコア:


  65 → z = -1.5000


...

  90 → z = 1.0000


  95 → z = 1.5000



標準化後のデータの性質:


  平均: 0.0000000000 (≈ 0)


  標準偏差: 1.0000000000 (≈ 1)


### 3.6.2 偏差値

In [18]:
// 偏差値 = 50 + 10 × Z
function toDeviationScore(value, mean, std) {
    const z = ss.zScore(value, mean, std);
    return 50 + 10 * z;
}

const examResults = [45, 55, 60, 65, 70, 75, 80, 85, 90];
const examMean = ss.mean(examResults);
const examStd = ss.standardDeviation(examResults);

console.log('試験結果:', examResults);
console.log('平均:', examMean.toFixed(2));
console.log('標準偏差:', examStd.toFixed(2));

console.log('\n偏差値:');
examResults.forEach(score => {
    const devScore = toDeviationScore(score, examMean, examStd);
    console.log(`  点数 ${score} → 偏差値 ${devScore.toFixed(1)}`);
});

試験結果: [
  45, 55, 60, 65, 70,
  75, 80, 85, 90
]


平均: 69.44


標準偏差: 13.83



偏差値:


  点数 45 → 偏差値 32.3


...

  点数 70 → 偏差値 50.4


  点数 75 → 偏差値 54.0


  点数 80 → 偏差値 57.6


  点数 85 → 偏差値 61.2


  点数 90 → 偏差値 64.9


### 3.6.3 Min-Max正規化

In [19]:
// 0〜1の範囲に正規化
const originalData = [10, 20, 30, 40, 50];

const minValue = ss.min(originalData);
const maxValue = ss.max(originalData);

const normalized = originalData.map(x => (x - minValue) / (maxValue - minValue));

console.log('元のデータ:', originalData);
console.log('最小値:', minValue);
console.log('最大値:', maxValue);
console.log('\nMin-Max正規化後:', normalized);

元のデータ: [ 10, 20, 30, 40, 50 ]


最小値: 10


最大値: 50



Min-Max正規化後: [ 0, 0.25, 0.5, 0.75, 1 ]


## 3.7 実践例: クラスの成績分析

In [20]:
// クラスの数学と英語の点数
const mathScores = [78, 85, 92, 65, 88, 72, 95, 80, 77, 83, 90, 68, 75, 82, 88, 70, 79, 86, 74, 91];
const englishScores = [72, 80, 88, 70, 85, 68, 90, 78, 75, 80, 86, 65, 72, 78, 84, 68, 76, 82, 70, 88];

console.log('=== クラスの成績分析 ===');
console.log('生徒数:', mathScores.length);

// 数学の分析
console.log('\n【数学】');
console.log('平均:', ss.mean(mathScores).toFixed(2));
console.log('中央値:', ss.median(mathScores));
console.log('標準偏差:', ss.standardDeviation(mathScores).toFixed(2));
console.log('最高点:', ss.max(mathScores));
console.log('最低点:', ss.min(mathScores));

// 英語の分析
console.log('\n【英語】');
console.log('平均:', ss.mean(englishScores).toFixed(2));
console.log('中央値:', ss.median(englishScores));
console.log('標準偏差:', ss.standardDeviation(englishScores).toFixed(2));
console.log('最高点:', ss.max(englishScores));
console.log('最低点:', ss.min(englishScores));

// 相関分析
console.log('\n【相関分析】');
const mathEngCorr = ss.sampleCorrelation(mathScores, englishScores);
console.log('数学と英語の相関係数:', mathEngCorr.toFixed(4));
console.log('決定係数 R²:', (mathEngCorr ** 2).toFixed(4));

=== クラスの成績分析 ===


生徒数: 20



【数学】


平均: 80.90


中央値: 81


...

最高点: 90


最低点: 65



【相関分析】


数学と英語の相関係数: 0.9700


決定係数 R²: 0.9409


## 3.8 練習問題

### 練習1: 代表値の計算

以下のデータについて、平均、中央値、最頻値を計算してください。

`[5, 7, 7, 8, 9, 10, 10, 10, 12, 15]`

In [21]:
// ここに回答を書いてください



### 練習2: 相関係数

以下のデータの相関係数を計算し、相関の強さを解釈してください。

```
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y = [2.1, 3.9, 6.2, 7.8, 10.1, 12.0, 14.2, 15.9, 18.1, 20.0]
```

In [22]:
// ここに回答を書いてください



## まとめ

この章では以下を学びました：

1. **代表値**: 算術平均、中央値、最頻値、幾何平均、調和平均
2. **散布度**: 範囲、分散、標準偏差、四分位範囲、変動係数
3. **分布の形状**: 歪度、尖度
4. **相関分析**: 共分散、ピアソンの相関係数、決定係数
5. **データの標準化**: Zスコア、偏差値、Min-Max正規化

次章では、確率分布について学びます。